In [2]:
from datasets import load_dataset

dataset = load_dataset("nexar-ai/nexar_collision_prediction", split="train")

print(dataset.cache_files)
print(dataset.column_names)
print(len(dataset))

train_valid = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_valid['train']
valid_dataset = train_valid['test']
print("train length:", len(train_dataset))
print("valid length:", len(valid_dataset))

c:\Users\USER\miniconda3\envs\accident_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'filename': 'C:\\Users\\USER\\.cache\\huggingface\\datasets\\nexar-ai___nexar_collision_prediction\\default\\0.0.0\\aa97deda5a59f00bb7187739053b7c72e14374df\\nexar_collision_prediction-train.arrow'}]
['video', 'time_of_event', 'time_of_alert', 'light_conditions', 'weather', 'scene', 'time_to_accident']
1500
train length: 1200
valid length: 300


In [4]:
test_dataset  = load_dataset("nexar-ai/nexar_collision_prediction", split="test")
print(len(test_dataset))

Using the latest cached version of the dataset since nexar-ai/nexar_collision_prediction couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\USER\.cache\huggingface\datasets\nexar-ai___nexar_collision_prediction\default\0.0.0\aa97deda5a59f00bb7187739053b7c72e14374df (last modified on Mon Oct 13 12:29:49 2025).


1344


In [5]:
import cv2
import numpy as np

def sample_frames(video_path, time_of_event=None, num_frames=16, seconds_before=7, seconds_after=7, size=(224,224)):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frames = []
    
    if time_of_event is not None and not np.isnan(time_of_event):
        if (time_of_event - seconds_before) < 0:
            start_time = 0
        else:
            start_time = time_of_event - seconds_before

        if (time_of_event + seconds_after) > (n_frames / fps):
            end_time = n_frames / fps
        else:
            end_time = time_of_event + seconds_after

        start_frame = int(start_time * fps)
        end_frame = int(end_time * fps)
        selected_frame = np.linspace(start_frame, end_frame, num_frames, dtype=int)
    else:
        selected_frame = np.linspace(0, n_frames - 1, num_frames, dtype=int)
    
    for i in selected_frame:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        flag, frame = cap.read()
        if not flag:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, size)
        frames.append(frame)
    
    cap.release()
    return np.array(frames)